### .rda 파일 읽기
### .rds 형식으로 변환
### pandas DataFrame 형식으로 변환

In [2]:
import pyreadr
import rpy2.robjects as ro
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

# .rda 파일 읽기
result = pyreadr.read_r('./data/wine.rda')

data = result['wine'].iloc[:, 1:].values
label = result['wine'].iloc[:,0].values

n = data.shape[0] # the number of samples
B = 5 # the number of bootstrapping
nk = 3 # the number of clusters
r = 5 # iteration for the kmeans
print("Number of samples: {}\nNumber of clusters: {}\nIteration for the kmeans: {}".format(n, nk, r))

clst_mat = np.empty((B + 1, n)) # cluster label for all samples for each bootstrapping
stab_matrix = np.empty((n, B)) # stability matrix (observation-wise stability)

# Clustering for the original data
km = KMeans(n_clusters=nk, max_iter=r)
km.fit(data)

center = km.cluster_centers_ # get cluster centroids
clst = km.labels_ # get cluster labels for the each sample

clst_mat[0, :] = clst # put the cluster label of original dataset

# Clustering for the bootstrapping data 
for b in range(B):
    resample = np.random.choice(n, size=n, replace=True) # index for random bootstrapping
    data_star = data[resample, :] # resampled data

    km_star = KMeans(n_clusters=nk, max_iter=r)
    km_star.fit(data_star)

    center_star = km_star.cluster_centers_ # cluster centroids of current bootstrap dataset
    class_star = cdist(center_star, data, metric='euclidean').argmin(axis=0) # original data와 bootstrap data에서 구한 centroids 중, 가장 거리가 짧은 centroid가 속한 cluster로 labeling
    clst_mat[b + 1, :] = class_star
    
    
class stability:
    def __init__(self, obs=0, clust=0, overall=0, ref_clust=0) -> None:
        self.obs = obs
        self.clust = clust
        self.overall = overall
        self.ref_clust = ref_clust

B1 = B + 1
agree_mat = np.empty((B1, B1)) # 
np.fill_diagonal(agree_mat, 1)

for i in range(B1 - 1):
    for j in range(i + 1, B1):
        agree_mat[i, j] = np.mean(clst_mat[i, :] == clst_mat[j, :]) # bootstrap_i와 bootstrap_j의 label 비교
        # --> 전체 중, 얼마나 일치하는지를 비율로 나타냄 (min=0; 모두 다른 경우, max=1; 모두 같은 경우)
        # --> bootstrap_i와 다른 모든 bootstrap 간의 유사도를 측정
        agree_mat[j, i] = agree_mat[i, j]

mean_agr = np.mean(agree_mat, axis=1) # 각 bootstrap이 다른 bootstrap과 가지는 유사도를 평균
ref = np.argmax(mean_agr) # --> 이 중, 평균치가 가장 높은 bootstrap data의 index 획득
# --> 유사도의 평균치가 가장 높은 bootstrap dataset의 cluster label이 B+1개의 dataset을 대표하는 label이라고 할 수 있음
clst_ref = clst_mat[ref, :] # --> 해당 bootstrap dataset의 label을 reference로 지정

stab_mat = np.empty((B1, n))

for i in range(B1):
    stab_mat[i, :] = (clst_ref == clst_mat[i, :])

ref_clust = clst_mat[0, ]
obs_wise = np.mean(stab_mat, axis=0)
clust_wise = []

for ki in range(nk):
    ki_clust_stab = obs_wise[ref_clust == ki]
    clust_wise.append(ki_clust_stab.mean())

overall = np.mean(clust_wise)

_stability = stability()
_stability.ref_clust = ref_clust
_stability.obs = obs_wise
_stability.clust = clust_wise
_stability.overall = overall

Number of samples: 178
Number of clusters: 3
Iteration for the kmeans: 5
